In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Import Data

In [2]:
data = pd.read_excel('Data_Train--Delivery.xlsx',usecols = [0,1,2,3,4,5,6,7])

### EDA & Preprocessing

In [3]:
data.head()

,Restaurant,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
0,ID_6321,"Fast Food, Rolls, Burger, Salad, Wraps",200,50,3.5,12,4,30
1,ID_2882,"Ice Cream, Desserts",100,50,3.5,11,4,30
2,ID_1595,"Italian, Street Food, Fast Food",150,50,3.6,99,30,65
3,ID_5929,"Mughlai, North Indian, Chinese",250,99,3.7,176,95,30
4,ID_6123,"Cafe, Beverages",200,99,3.2,521,235,65


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7772 entries, 0 to 7771
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Restaurant     7772 non-null   object 
 1   Cuisines       7772 non-null   object 
 2   Average_Cost   7772 non-null   int64  
 3   Minimum_Order  7772 non-null   int64  
 4   Rating         7772 non-null   float64
 5   Votes          7772 non-null   int64  
 6   Reviews        7772 non-null   int64  
 7   Delivery_Time  7772 non-null   int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 485.9+ KB


In [5]:
#Label encoding categorical variables
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
data['Restaurant_Code'] = lb_make.fit_transform(data['Restaurant'])

#Check that encoding is correct 
#data.loc[data.duplicated(subset=['Restaurant_Code'])].sort_values(by=['Restaurant_Code'])

# creates X & Y
x_e = data.drop(columns = ['Delivery_Time','Restaurant','Cuisines'])
y_e = data.Delivery_Time

x_e.shape

(7772, 6)

In [6]:
#Binary encoding
import category_encoders as ce

encoder = ce.BinaryEncoder(cols=['Restaurant_Code'])

x_b = encoder.fit_transform(x_e)#.drop(columns='Restaurant_Code')

x_b.shape

(7772, 18)

### SVM

In [7]:
from sklearn.svm import SVR

from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV

seed = 123

In [8]:
#Search for best parameters

params_to_search = {'C':[0.1,1,10,100,1000], 'gamma':[1,0.1,0.001,0.0001]}

grid = GridSearchCV(SVR(), params_to_search, verbose = 3, n_jobs = -1)

grid.fit(x_b,y_e)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


GridSearchCV(estimator=SVR(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.001, 0.0001]},
             verbose=3)

In [9]:
grid.best_params_

{'C': 1, 'gamma': 0.001}

In [17]:
grid

GridSearchCV(estimator=SVR(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.001, 0.0001]},
             verbose=3)

In [13]:
svr = SVR(C= 1, gamma = 0.001)

In [14]:
svr_model = svr.fit(x_b,y_e)

In [15]:
#Validation

metric = 'neg_root_mean_squared_error'

kfold_dtr = cross_val_score(svr ,x_b ,y_e ,cv = 10, n_jobs = -1,scoring = metric)

In [16]:
#Metric results

print('Support Vector Regressor RMSE:',np.absolute(kfold_dtr.mean()))

Support Vector Regressor RMSE: 10.432348865408139
